In [9]:
import pandas as pd
import numpy as np
from fmiopendata.wfs import download_stored_query
import datetime as dt
import os

#### This is for one data file

We fill all Nan values by using the geographically closest stations, fit a 1d polynomial between the closest and the current station and apply the fitted function to the closest station values to fill the Nan values.

In [15]:
# Assuming 'df' is your DataFrame loaded from a CSV file
# Replace 'your_data.csv' with the path to your actual CSV file
file_path='data_by_measurement_type/air_temperature_data.csv'

timestamp_column = 'Timestamp'

# Read the CSV file, setting the timestamp column as the index
df = pd.read_csv(file_path, index_col=0, parse_dates=True)

missing_values = df.isnull().sum()

print("Missing values in each column:")
print(missing_values)


Missing values in each column:
Alajärvi Möksy                         1
Asikkala Pulkkilanharju                7
Eckerö Torp                         3910
Enontekiö Kilpisjärvi Saana          253
Enontekiö Kilpisjärvi kyläkeskus     254
                                    ... 
Virolahti Koivuniemi                   1
Virrat Äijänneva                       6
Ylitornio Meltosjärvi                 42
Ylivieska lentokenttä                 22
Ähtäri Inha                            0
Length: 223, dtype: int64


In [16]:

# Function to find the closest station with non-NaN values
def find_closest_station(current_station, stations_metadata, df, excluded_stations):
    min_distance = float('inf')
    closest_station = None
    for station, metadata in stations_metadata.items():
        if station != current_station and station not in excluded_stations and not df[station].isnull().all():
            distance = np.sqrt((metadata['longitude'] - stations_metadata[current_station]['longitude'])**2 +
                               (metadata['latitude'] - stations_metadata[current_station]['latitude'])**2)
            if distance < min_distance:
                min_distance = distance
                closest_station = station
    return closest_station

# Function to fill NaN values using polyfit from the closest station
def fill_nan_values(station, stations_metadata, df):
    nan_indices = df[station][df[station].isnull()].index
    excluded_stations = []
    
    while len(nan_indices) > 0:
        closest_station_name = find_closest_station(station, stations_metadata, df, excluded_stations)
        print(closest_station_name)
        
        if closest_station_name is None:  # No more stations to check
            break
            
        excluded_stations.append(closest_station_name)
        series_a = df[station]
        series_b = df[closest_station_name]

        # Find indices where either series A or B has NaN values
        nan_indices = series_a[series_a.isnull() | series_b.isnull()].index

        # Remove the NaN values from both series
        clean_a = series_a.drop(nan_indices)
        clean_b = series_b.drop(nan_indices)
        if len(clean_a)==0 or  len(clean_b)==0:
            return

        # Ensure both series are of the same length after dropping NaN values
        assert len(clean_a) == len(clean_b), "The series are not of the same length after dropping NaN values."

        # Perform polynomial fit on the cleaned data
        coeffs = np.polyfit(clean_a, clean_b, 1)  # Degree can be adjusted
        poly = np.poly1d(coeffs)

        # Fill NaN values in station A using the polynomial function and values from station B
        for idx in df[station][df[station].isnull()].index:
            if not np.isnan(df.at[idx, closest_station_name]):
                filled_value = round(poly(df.at[idx, closest_station_name]), 2)
                df.at[idx, station] = filled_value
                
        nan_indices = df[station][df[station].isnull()].index
        print(f"Filled NaN values for station {station} using {closest_station_name}. Remaining NaN count: {len(nan_indices)}")

    

# Retrieve station metadata
end_time = dt.datetime.utcnow()
start_time = end_time - dt.timedelta(hours=1)
obs = download_stored_query("fmi::observations::weather::multipointcoverage",
                            args=["bbox=18,55,35,75",
                                  "starttime=" + start_time.isoformat(timespec="seconds") + "Z",
                                  "endtime=" + end_time.isoformat(timespec="seconds") + "Z"])

stations_metadata = {station: metadata for station, metadata in obs.location_metadata.items()}

# Main loop to fill NaN values for each station
for station in df.columns:
    if station in stations_metadata:
        fill_nan_values(station, stations_metadata, df)


Halsua Purola
Filled NaN values for station Alajärvi Möksy using Halsua Purola. Remaining NaN count: 0
Lahti Sopenkorpi
Filled NaN values for station Asikkala Pulkkilanharju using Lahti Sopenkorpi. Remaining NaN count: 7
Luhanka Judinsalo
Filled NaN values for station Asikkala Pulkkilanharju using Luhanka Judinsalo. Remaining NaN count: 6
Hämeenlinna Lammi Pappila
Filled NaN values for station Asikkala Pulkkilanharju using Hämeenlinna Lammi Pappila. Remaining NaN count: 0
Enontekiö Kilpisjärvi kyläkeskus
Filled NaN values for station Enontekiö Kilpisjärvi Saana using Enontekiö Kilpisjärvi kyläkeskus. Remaining NaN count: 253
Enontekiö lentoasema
Filled NaN values for station Enontekiö Kilpisjärvi Saana using Enontekiö lentoasema. Remaining NaN count: 252
Enontekiö Näkkälä
Filled NaN values for station Enontekiö Kilpisjärvi Saana using Enontekiö Näkkälä. Remaining NaN count: 252
Muonio Oustajärvi
Filled NaN values for station Enontekiö Kilpisjärvi Saana using Muonio Oustajärvi. Remainin

Filled NaN values for station Ilmajoki Seinäjoki lentoasema using Seinäjoki Pelmaa. Remaining NaN count: 1
Kauhava lentokenttä
Filled NaN values for station Ilmajoki Seinäjoki lentoasema using Kauhava lentokenttä. Remaining NaN count: 1
Karvia Alkkia
Filled NaN values for station Ilmajoki Seinäjoki lentoasema using Karvia Alkkia. Remaining NaN count: 0
Ilomantsi Mekrijärvi
Filled NaN values for station Ilomantsi Pötsönvaara using Ilomantsi Mekrijärvi. Remaining NaN count: 0
Kittilä Pokka
Filled NaN values for station Inari Angeli Lintupuoliselkä using Kittilä Pokka. Remaining NaN count: 245
Kittilä Pakatti
Filled NaN values for station Inari Angeli Lintupuoliselkä using Kittilä Pakatti. Remaining NaN count: 245
Kittilä lentoasema
Filled NaN values for station Inari Angeli Lintupuoliselkä using Kittilä lentoasema. Remaining NaN count: 245
Utsjoki Kevo
Filled NaN values for station Inari Angeli Lintupuoliselkä using Utsjoki Kevo. Remaining NaN count: 245
Utsjoki Kevo Kevojärvi
Filled NaN

Kristiinankaupunki Majakka
Filled NaN values for station Kaskinen Sälgrund using Kristiinankaupunki Majakka. Remaining NaN count: 0
Seinäjoki Pelmaa
Filled NaN values for station Kauhajoki Kuja-Kokko using Seinäjoki Pelmaa. Remaining NaN count: 1
Kankaanpää Niinisalo lentokenttä
Filled NaN values for station Kauhajoki Kuja-Kokko using Kankaanpää Niinisalo lentokenttä. Remaining NaN count: 0
Ilmajoki Seinäjoki lentoasema
Filled NaN values for station Kauhava lentokenttä using Ilmajoki Seinäjoki lentoasema. Remaining NaN count: 0
Kemi Kemi-Tornio lentoasema
Filled NaN values for station Kemi Ajos using Kemi Kemi-Tornio lentoasema. Remaining NaN count: 35
Tornio Torppi
Filled NaN values for station Kemi Ajos using Tornio Torppi. Remaining NaN count: 35
Kemi I majakka
Filled NaN values for station Kemi Ajos using Kemi I majakka. Remaining NaN count: 30
Hailuoto Marjaniemi
Filled NaN values for station Kemi Ajos using Hailuoto Marjaniemi. Remaining NaN count: 0
Tornio Torppi
Filled NaN valu

Kuusamo Rukatunturi
Filled NaN values for station Kuusamo Ruka Talvijärvi using Kuusamo Rukatunturi. Remaining NaN count: 35
Kuusamo Välikangas
Filled NaN values for station Kuusamo Ruka Talvijärvi using Kuusamo Välikangas. Remaining NaN count: 35
Kuusamo lentoasema
Filled NaN values for station Kuusamo Ruka Talvijärvi using Kuusamo lentoasema. Remaining NaN count: 35
Kuusamo Kiutaköngäs
Filled NaN values for station Kuusamo Ruka Talvijärvi using Kuusamo Kiutaköngäs. Remaining NaN count: 0
Kuusamo Ruka Talvijärvi
Filled NaN values for station Kuusamo Rukatunturi using Kuusamo Ruka Talvijärvi. Remaining NaN count: 0
Kuusamo lentoasema
Filled NaN values for station Kuusamo Välikangas using Kuusamo lentoasema. Remaining NaN count: 201
Kuusamo Rukatunturi
Filled NaN values for station Kuusamo Välikangas using Kuusamo Rukatunturi. Remaining NaN count: 0
Kuusamo Välikangas
Filled NaN values for station Kuusamo lentoasema using Kuusamo Välikangas. Remaining NaN count: 0
Lemland Nyhamn
Filled 

Filled NaN values for station Rauma Pyynpää using Rauma Kylmäpihlaja. Remaining NaN count: 0
Rovaniemi lentoasema AWOS
Filled NaN values for station Rovaniemi Apukka using Rovaniemi lentoasema AWOS. Remaining NaN count: 42
Rovaniemi lentoasema
Filled NaN values for station Rovaniemi Apukka using Rovaniemi lentoasema. Remaining NaN count: 42
Rovaniemi rautatieasema
Filled NaN values for station Rovaniemi Apukka using Rovaniemi rautatieasema. Remaining NaN count: 42
Ranua lentokenttä
Filled NaN values for station Rovaniemi Apukka using Ranua lentokenttä. Remaining NaN count: 0
Rovaniemi lentoasema AWOS
Filled NaN values for station Rovaniemi lentoasema using Rovaniemi lentoasema AWOS. Remaining NaN count: 43
Rovaniemi rautatieasema
Filled NaN values for station Rovaniemi lentoasema using Rovaniemi rautatieasema. Remaining NaN count: 42
Rovaniemi Apukka
Filled NaN values for station Rovaniemi lentoasema using Rovaniemi Apukka. Remaining NaN count: 0
Rovaniemi lentoasema
Filled NaN values 

In [17]:
# check if successful

In [18]:
missing_values = df.isnull().sum()

print("Missing values in each column:")
print(missing_values)

Missing values in each column:
Alajärvi Möksy                         0
Asikkala Pulkkilanharju                0
Eckerö Torp                         3910
Enontekiö Kilpisjärvi Saana            0
Enontekiö Kilpisjärvi kyläkeskus       0
                                    ... 
Virolahti Koivuniemi                   0
Virrat Äijänneva                       0
Ylitornio Meltosjärvi                  0
Ylivieska lentokenttä                  0
Ähtäri Inha                            0
Length: 223, dtype: int64


In [19]:
#write df back to file

In [23]:
try:
    os.makedirs('infilled_data/data_by_measurement_type/')
except:
    pass
output_file_path = 'infilled_data/data_by_measurement_type/air_temperature_data.csv.csv'

# Write the DataFrame to a CSV file
df.to_csv(output_file_path, sep=',', index=True)

print(f"DataFrame written to '{output_file_path}'")

DataFrame written to 'infilled_data/data_by_measurement_type/air_temperature_data.csv.csv'


### This does all files

In [14]:
import os
import glob
import pandas as pd
import numpy as np
from fmiopendata.wfs import download_stored_query
import datetime as dt

def read_and_preprocess(file_path):
    df = pd.read_csv(file_path, index_col=0, parse_dates=True)
    missing_values = df.isnull().sum()
    print("Missing values in each column:")
    print(missing_values)
    return df

def retrieve_station_metadata():
    end_time = dt.datetime.utcnow()
    start_time = end_time - dt.timedelta(hours=1)
    obs = download_stored_query("fmi::observations::weather::multipointcoverage",
                                args=["bbox=18,55,35,75",
                                      "starttime=" + start_time.isoformat(timespec="seconds") + "Z",
                                      "endtime=" + end_time.isoformat(timespec="seconds") + "Z"])
    return {station: metadata for station, metadata in obs.location_metadata.items()}

def process_file(file_path, stations_metadata):
    df = read_and_preprocess(file_path)
    for station in df.columns:
        if station in stations_metadata:
            fill_nan_values(station, stations_metadata, df)
    return df

def write_to_disk(df, output_file_path):
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    df.to_csv(output_file_path, sep=',', index=True)
    print(f"DataFrame written to '{output_file_path}'")

# Functions find_closest_station and fill_nan_values as you defined them go here

# Main logic to process all files in a folder
data_folder_path = 'data_by_measurement_type'
output_folder_path = 'infilled_data/data_by_measurement_type'

stations_metadata = retrieve_station_metadata()

for file_name in glob.glob(os.path.join(data_folder_path, '*.csv')):
    print(f"Processing file: {file_name}")
    processed_df = process_file(file_name, stations_metadata)
    output_file_name = os.path.basename(file_name).replace('.csv', '_filled.csv')
    output_file_path = os.path.join(output_folder_path, output_file_name)
    write_to_disk(processed_df, output_file_path)

Processing file: data_by_measurement_type/precipitation_amount_data.csv
Missing values in each column:
Alajärvi Möksy                         2
Asikkala Pulkkilanharju             1483
Eckerö Torp                         1483
Enontekiö Kilpisjärvi Saana         1483
Enontekiö Kilpisjärvi kyläkeskus       7
                                    ... 
Virolahti Koivuniemi                   4
Virrat Äijänneva                    1483
Ylitornio Meltosjärvi                  0
Ylivieska lentokenttä                  0
Ähtäri Inha                            0
Length: 223, dtype: int64
Ähtäri Inha
Filled NaN values for station Alajärvi Möksy using Ähtäri Inha. Remaining NaN count: 0
Lahti Sopenkorpi
Enontekiö Kilpisjärvi kyläkeskus
Enontekiö Näkkälä
Filled NaN values for station Enontekiö Kilpisjärvi kyläkeskus using Enontekiö Näkkälä. Remaining NaN count: 0
Muonio Oustajärvi
Filled NaN values for station Enontekiö Näkkälä using Muonio Oustajärvi. Remaining NaN count: 0
Enontekiö Näkkälä
Nurmijärvi

Filled NaN values for station Savukoski Tulppio using Salla Värriötunturi. Remaining NaN count: 0
Sodankylä Lokka
Filled NaN values for station Savukoski kirkonkylä using Sodankylä Lokka. Remaining NaN count: 1
Salla kirkonkylä
Filled NaN values for station Savukoski kirkonkylä using Salla kirkonkylä. Remaining NaN count: 0
Kauhava lentokenttä
Filled NaN values for station Seinäjoki Pelmaa using Kauhava lentokenttä. Remaining NaN count: 0
Kuopio Savilahti
Helsinki Kumpula
Sodankylä Vuotso
Filled NaN values for station Sodankylä Lokka using Sodankylä Vuotso. Remaining NaN count: 0
Kemijärvi lentokenttä
Filled NaN values for station Sodankylä Tähtelä using Kemijärvi lentokenttä. Remaining NaN count: 0
Inari Saariselkä matkailukeskus
Filled NaN values for station Sodankylä Vuotso using Inari Saariselkä matkailukeskus. Remaining NaN count: 0
Salo Kiikala lentokenttä
Filled NaN values for station Somero Salkola using Salo Kiikala lentokenttä. Remaining NaN count: 0
Sotkamo Kuolaniemi
Filled

Hanko Russarö
Hämeenlinna Katinen
Asikkala Pulkkilanharju
Helsinki Kaisaniemi
Filled NaN values for station Helsinki Harmaja using Helsinki Kaisaniemi. Remaining NaN count: 0
Helsinki Harmaja
Filled NaN values for station Helsinki Helsingin majakka using Helsinki Harmaja. Remaining NaN count: 0
Helsinki Kumpula
Filled NaN values for station Helsinki Kaisaniemi using Helsinki Kumpula. Remaining NaN count: 0
Helsinki Kaisaniemi
Filled NaN values for station Helsinki Kumpula using Helsinki Kaisaniemi. Remaining NaN count: 0
Helsinki Kumpula
Filled NaN values for station Helsinki Malmi lentokenttä using Helsinki Kumpula. Remaining NaN count: 0
Helsinki Vuosaari satama
Sipoo Itätoukki
Filled NaN values for station Helsinki Vuosaari satama using Sipoo Itätoukki. Remaining NaN count: 218
Helsinki Malmi lentokenttä
Filled NaN values for station Helsinki Vuosaari satama using Helsinki Malmi lentokenttä. Remaining NaN count: 0
Vantaa Helsinki-Vantaan lentoasema
Vihti Maasoja
Filled NaN values fo

Filled NaN values for station Jyväskylä lentoasema AWOS using Jyväskylä lentoasema. Remaining NaN count: 0
Jämsä Halli lentoasema
Multia Karhila
Filled NaN values for station Jämsä Halli lentoasema using Multia Karhila. Remaining NaN count: 0
Mäntsälä Hirvihaara
Kemiönsaari Kemiö
Filled NaN values for station Kaarina Yltöinen using Kemiönsaari Kemiö. Remaining NaN count: 13
Turku lentoasema
Filled NaN values for station Kaarina Yltöinen using Turku lentoasema. Remaining NaN count: 1
Turku Artukainen
Filled NaN values for station Kaarina Yltöinen using Turku Artukainen. Remaining NaN count: 0
Kajaani lentoasema
Filled NaN values for station Kajaani Petäisenniska using Kajaani lentoasema. Remaining NaN count: 13
Sotkamo Tuhkakylä
Filled NaN values for station Kajaani Petäisenniska using Sotkamo Tuhkakylä. Remaining NaN count: 13
Puolanka Paljakka
Filled NaN values for station Kajaani Petäisenniska using Puolanka Paljakka. Remaining NaN count: 0
Kajaani Petäisenniska
Filled NaN values for

Filled NaN values for station Loviisa Orrengrund using Kotka Rankki. Remaining NaN count: 0
Asikkala Pulkkilanharju
Filled NaN values for station Luhanka Judinsalo using Asikkala Pulkkilanharju. Remaining NaN count: 0
Jomala Jomalaby
Filled NaN values for station Lumparland Långnäs satama using Jomala Jomalaby. Remaining NaN count: 0
Korsnäs Bredskäret
Filled NaN values for station Maalahti Strömmingsbådan using Korsnäs Bredskäret. Remaining NaN count: 0
Maarianhamina Länsisatama
Filled NaN values for station Maarianhamina Lotsberget using Maarianhamina Länsisatama. Remaining NaN count: 248
Jomala Maarianhamina lentoasema
Filled NaN values for station Maarianhamina Lotsberget using Jomala Maarianhamina lentoasema. Remaining NaN count: 0
Maarianhamina Lotsberget
Filled NaN values for station Maarianhamina Länsisatama using Maarianhamina Lotsberget. Remaining NaN count: 0
Mikkeli lentoasema
Filled NaN values for station Mikkeli Lentoasema AWOS using Mikkeli lentoasema. Remaining NaN coun

...

Filled NaN values for station Helsinki Malmi lentokenttä using Helsinki Kumpula. Remaining NaN count: 0
Helsinki Vuosaari satama
Sipoo Itätoukki
Filled NaN values for station Helsinki Vuosaari satama using Sipoo Itätoukki. Remaining NaN count: 259
Helsinki Malmi lentokenttä
Filled NaN values for station Helsinki Vuosaari satama using Helsinki Malmi lentokenttä. Remaining NaN count: 0
Nurmijärvi Röykkä
Filled NaN values for station Hyvinkää Hyvinkäänkylä using Nurmijärvi Röykkä. Remaining NaN count: 0
Hattula Lepaa
Filled NaN values for station Hämeenlinna Katinen using Hattula Lepaa. Remaining NaN count: 0
Mäntsälä Hirvihaara
Filled NaN values for station Hämeenlinna Lammi Pappila using Mäntsälä Hirvihaara. Remaining NaN count: 0
Seinäjoki Pelmaa
Filled NaN values for station Ilmajoki Seinäjoki lentoasema using Seinäjoki Pelmaa. Remaining NaN count: 10
Kauhava lentokenttä
Filled NaN values for station Ilmajoki Seinäjoki lentoasema using Kauhava lentokenttä. Remaining NaN count: 1
Karvi

Jomala Maarianhamina lentoasema
Filled NaN values for station Jomala Jomalaby using Jomala Maarianhamina lentoasema. Remaining NaN count: 0
Maarianhamina Länsisatama
Filled NaN values for station Jomala Maarianhamina lentoasema using Maarianhamina Länsisatama. Remaining NaN count: 11
Jomala Jomalaby
Filled NaN values for station Jomala Maarianhamina lentoasema using Jomala Jomalaby. Remaining NaN count: 0
Luhanka Judinsalo
Filled NaN values for station Joutsa Savenaho using Luhanka Judinsalo. Remaining NaN count: 0
Rautavaara Ylä-Luosta
Filled NaN values for station Juuka Niemelä using Rautavaara Ylä-Luosta. Remaining NaN count: 0
Jämsä Halli lentoasema
Filled NaN values for station Juupajoki Hyytiälä using Jämsä Halli lentoasema. Remaining NaN count: 0
Varkaus Kosulanniemi
Filled NaN values for station Juva Partala using Varkaus Kosulanniemi. Remaining NaN count: 6
Puumala kirkonkylä
Filled NaN values for station Juva Partala using Puumala kirkonkylä. Remaining NaN count: 6
Rantasalmi

Siilinjärvi Kuopio lentoasema
Filled NaN values for station Kuopio Ritoniemi using Siilinjärvi Kuopio lentoasema. Remaining NaN count: 0
Rauma Kylmäpihlaja
Filled NaN values for station Kustavi Isokari using Rauma Kylmäpihlaja. Remaining NaN count: 2
Kumlinge kirkonkylä
Filled NaN values for station Kustavi Isokari using Kumlinge kirkonkylä. Remaining NaN count: 0
Kuusamo Kiutaköngäs
Filled NaN values for station Kuusamo Juuma using Kuusamo Kiutaköngäs. Remaining NaN count: 35
Kuusamo Rukatunturi
Filled NaN values for station Kuusamo Juuma using Kuusamo Rukatunturi. Remaining NaN count: 35
Kuusamo Ruka Talvijärvi
Filled NaN values for station Kuusamo Juuma using Kuusamo Ruka Talvijärvi. Remaining NaN count: 35
Kuusamo Välikangas
Filled NaN values for station Kuusamo Juuma using Kuusamo Välikangas. Remaining NaN count: 35
Kuusamo lentoasema
Filled NaN values for station Kuusamo Juuma using Kuusamo lentoasema. Remaining NaN count: 35
Salla kirkonkylä
Filled NaN values for station Kuusamo

Kokkola Tankar
Filled NaN values for station Pietarsaari Kallan using Kokkola Tankar. Remaining NaN count: 0
Tampere Härmälä
Filled NaN values for station Pirkkala Tampere-Pirkkala lentoasema using Tampere Härmälä. Remaining NaN count: 1
Tampere Siilinkari
Filled NaN values for station Pirkkala Tampere-Pirkkala lentoasema using Tampere Siilinkari. Remaining NaN count: 0
Pori rautatieasema
Filled NaN values for station Pori Tahkoluoto satama using Pori rautatieasema. Remaining NaN count: 0
Pori rautatieasema
Filled NaN values for station Pori lentoasema using Pori rautatieasema. Remaining NaN count: 6
Rauma Pyynpää
Filled NaN values for station Pori lentoasema using Rauma Pyynpää. Remaining NaN count: 6
Pori Tahkoluoto satama
Filled NaN values for station Pori lentoasema using Pori Tahkoluoto satama. Remaining NaN count: 0
Pori lentoasema
Filled NaN values for station Pori rautatieasema using Pori lentoasema. Remaining NaN count: 0
Porvoo Kilpilahti satama
Filled NaN values for station 

Utsjoki Kevo Kevojärvi
Filled NaN values for station Utsjoki Kevo using Utsjoki Kevo Kevojärvi. Remaining NaN count: 252
Inari Kaamanen
Filled NaN values for station Utsjoki Kevo using Inari Kaamanen. Remaining NaN count: 0
Utsjoki Kevo
Filled NaN values for station Utsjoki Kevo Kevojärvi using Utsjoki Kevo. Remaining NaN count: 0
Utsjoki Kevo Kevojärvi
Filled NaN values for station Utsjoki Nuorgam using Utsjoki Kevo Kevojärvi. Remaining NaN count: 0
Oulu Kaukovainio
Filled NaN values for station Vaala Pelso using Oulu Kaukovainio. Remaining NaN count: 0
Vaasa lentoasema
Filled NaN values for station Vaasa Klemettilä using Vaasa lentoasema. Remaining NaN count: 13
Korsnäs Bredskäret
Filled NaN values for station Vaasa Klemettilä using Korsnäs Bredskäret. Remaining NaN count: 0
Vaasa Klemettilä
Filled NaN values for station Vaasa lentoasema using Vaasa Klemettilä. Remaining NaN count: 0
Helsinki Malmi lentokenttä
Filled NaN values for station Vantaa Helsinki-Vantaan lentoasema using Hel